In [1]:
import requests
from pathlib import Path
import xmltodict
import os
import pandas as pd
import io
import datetime
import pymongo
import json
import gzip
import dns
from os import listdir
from os.path import isfile, join

urlBase='https://tisvcloud.freeway.gov.tw/history/vd/'  # 20190624/cms_value_0000.xml.gz   20190624/cms_value_0014.xml.gz
baseDir='../../../data/'
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.traffic
trType='vd'
collect = db['vd5']

In [2]:
#確認該 url 是否可以下載, 例如 20190230 就不存在, 無法下載; 而且必須是附件形式的方可
def isDownloadable(url):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if content_type is None:
        return False
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return True


#下載某url檔案後, 放在指定目錄下
def downloadFileFromUrl(url, directory):
    filename = directory+'/'+ url.rsplit('/', 1)[1]
    if not os.path.exists(filename):
        r = requests.get(url, allow_redirects=True)
        open(filename, 'wb').write(r.content)

In [3]:
#抓取 CMS 的某一天每1分鐘一次 所有檔案 並下載到指定目錄
def downloadVD1MinDay(trType, day, baseDir):
    downloads=[]
    for hour in range(0,24):
        for mininute in range(0,60,1):  #vd 每5分鐘一次
            downloads.append(urlBase + day + '/' + trType + '_value_'+ format(hour, '02d')+ format(mininute, '02d') +'.xml.gz' )
    p=baseDir+trType+'1Min'+'/'+day
    Path(p).mkdir(parents=True, exist_ok=True)
    for url in downloads:
        if isDownloadable(url):
            downloadFileFromUrl(url,p)
            #print(url)  

In [5]:
#抓取 CMS 的某一天每5分鐘一次 所有檔案 並下載到指定目錄
def downloadVD5MinDay(trType, day, baseDir):
    downloads=[]
    for hour in range(0,24):
        for mininute in range(0,60,5):  #vd 每5分鐘一次
            downloads.append(urlBase + day + '/' + trType + '_value5_'+ format(hour, '02d')+ format(mininute, '02d') +'.xml.gz' )
    p=baseDir+trType+'/'+day
    Path(p).mkdir(parents=True, exist_ok=True)
    for url in downloads:
        if isDownloadable(url):
            downloadFileFromUrl(url,p)
            #print(url)  

In [6]:
from pandas import json_normalize #package for flattening json in pandas df

def splitLane2DF(lanes):
    tmpDF=json_normalize(data=lanes, record_path='cars', meta=['@vsrid', '@speed','@laneoccupy'])
    records = tmpDF.to_dict('records')
    return records
 

In [7]:
          
def getAllFiles2Mongo(trType, day, baseDir, OneMinute):
    if OneMinute:
        directory=baseDir+trType+'1Min'+'/'+day+'/'
        collect = db['vd1']
    else:
        directory=baseDir+trType+'/'+day+'/'
    
    files = [f for f in listdir(directory) if isfile(join(directory, f))]
    i=1
    for file in files:
        print(i,directory,file)
        i += 1
        f = gzip.open(directory+file, 'rb')
        doc = xmltodict.parse(f)
        Y=doc['XML_Head']['Infos']
        if Y is not None:
            X=Y['Info']
            #has datacollecttime, no need updateTime=datetime.datetime.strptime(doc['XML_Head']['@updatetime'], "%Y/%m/%d %H:%M:%S")
            json_data = json.dumps(X)
            cc = json.loads(json_data)
            tmpDF=pd.DataFrame(cc)
            tmpDF['@datacollecttime']=tmpDF['@datacollecttime'].apply(lambda x:datetime.datetime.strptime(x, "%Y/%m/%d %H:%M:%S"))
            tmpDF['lane']=tmpDF['lane'].apply(splitLane2DF)
            #print(tmpDF['lane'][0],'\n',tmpDF['lane'][1],'\n',tmpDF['lane'][2],'\n',tmpDF['lane'][3])
            #a=cc[1]
            #print(a['@vdid'],a['@datacollecttime'],a['@status'],'\n',splitLane2DF(a['lane'])) 
            records = tmpDF.to_dict('records')
            try:
                collect.insert_many(records)
            except Exception as err:
                print ("collect.insert_many ERROR:", err)    
        else:
            print('TypeError: ',Y)
        
        f.close()
        ######3##########TODO 

In [8]:
def download5MinInsert(trType, day, baseDir):
    downloadVD5MinDay(trType, day, baseDir)
    getAllFiles2Mongo(trType, day, baseDir, False)

In [9]:
#days=['0405','0406','0407','0606','0607','0608'] #,'0624','0625','0626','0627','0628','0202','0203','0204','0205','0206','0207','0208','0209','0210']#
#days=['1010','1011','1012','1013','0913','0914','0915']
days=['0322','1210','1218']#,'0307']
for d in days:
    #downloadVD5MinDay('vd','2019'+d,baseDir)
    getAllFiles2Mongo('vd','2019'+d,baseDir, False)

1 ../../../data/vd/20190322/ vd_value5_0715.xml.gz
collect.insert_many ERROR: local variable 'collect' referenced before assignment
2 ../../../data/vd/20190322/ vd_value5_0025.xml.gz


KeyboardInterrupt: 

In [10]:
def download1MinInsert(trType, day, baseDir):
    downloadVD1MinDay(trType, day, baseDir)
    getAllFiles2Mongo(trType, day, baseDir, True)


In [12]:
days=['0322'] #,'1210','1218']#,'0307']
for d in days:
    download1MinInsert('vd','2019'+d,baseDir)

1 ../../../data/vd1Min/20190322/ vd_value_0311.xml.gz
2 ../../../data/vd1Min/20190322/ vd_value_0436.xml.gz
3 ../../../data/vd1Min/20190322/ vd_value_0535.xml.gz
4 ../../../data/vd1Min/20190322/ vd_value_0128.xml.gz
5 ../../../data/vd1Min/20190322/ vd_value_1517.xml.gz
6 ../../../data/vd1Min/20190322/ vd_value_0830.xml.gz
7 ../../../data/vd1Min/20190322/ vd_value_1811.xml.gz
8 ../../../data/vd1Min/20190322/ vd_value_0948.xml.gz
9 ../../../data/vd1Min/20190322/ vd_value_0551.xml.gz
10 ../../../data/vd1Min/20190322/ vd_value_1208.xml.gz
11 ../../../data/vd1Min/20190322/ vd_value_1826.xml.gz
12 ../../../data/vd1Min/20190322/ vd_value_0441.xml.gz
13 ../../../data/vd1Min/20190322/ vd_value_2234.xml.gz
14 ../../../data/vd1Min/20190322/ vd_value_1230.xml.gz
15 ../../../data/vd1Min/20190322/ vd_value_1034.xml.gz
16 ../../../data/vd1Min/20190322/ vd_value_2151.xml.gz
17 ../../../data/vd1Min/20190322/ vd_value_2301.xml.gz
18 ../../../data/vd1Min/20190322/ vd_value_1959.xml.gz
19 ../../../data/vd